In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1
)

### from_template() 메소드
* 변수를 중괄호로 묶어서 템플릿에 정의하는 방법을 사용

In [3]:
from langchain_core.prompts import PromptTemplate

# 템플릿 정의, {}안의 내용은 이후에 값이 들어갈 자리
template = "{name}의 직업은 무엇인가요?"

# from_template 메소드를 사용하여 PromptTemplate 객체를 생성
prompt = PromptTemplate.from_template(template)

print(prompt)

input_variables=['name'] input_types={} partial_variables={} template='{name}의 직업은 무엇인가요?'


In [5]:
# prompt 생성(완성), format() 메소드의 name 인자를 이용하여 변수에 값을 넣음
prompt = prompt.format(name="bear")

print(prompt)

bear의 직업은 무엇인가요?


In [10]:
from langchain_core.prompts import PromptTemplate

template = "{language}는 누가 만들었나요?"

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm

print(prompt)

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요?'


In [8]:
# print(chain.invoke({"language": "Python"}).content)
# 딕셔너리 형태로 변수를 작성하는게 원칙이지만 변수가 하나일때는 변수만으로 작성 가능
print(chain.invoke({"Python"}).content)

Python은 귀도 반 로썸(Guido van Rossum)에 의해 1989년에 개발되었습니다. 그는 1991년에 Python의 첫 번째 버전을 공개하였고, 이후 Python은 전 세계적으로 널리 사용되는 프로그래밍 언어로 성장하게 되었습니다.


### PromptTemplates 객체 생성과 동시에 prompt 생성
* input_variables 인자를 사용하여 변수를 지정한다.
* 템플릿에 작성한 변수가 input_variables에 없으면 예외를 발생시켜준다.

In [11]:
# template 정의
template = "{language}는 누가 만들었나요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["language"]
)

print(prompt)

prompt.format(language="Python")

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요?'


'Python는 누가 만들었나요?'

**partial_variables**
* 연산중 미리 계산된 변수를 프롬프트 템플릿에 지정해서 넣을 수 있다.
* 항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우 사용<br>ex)날짜, 시간

In [17]:
# template 정의
template = "{language1}과 {language2}는 각각 누가 만들었나요?"

# PromptTemplate 객체 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["language1"],
    partial_variables={
        "language2": "Java" # dictionary 형태로 partial_value 작성
    }
)

print(prompt)

input_variables=['language1'] input_types={} partial_variables={'language2': 'Java'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [18]:
prompt.format(language1="Python")

'Python과 Java는 각각 누가 만들었나요?'

**partial()**
* 연산중에 미리 계산된 변수를 프롬프트 템플릿에 지정해서 넣을 수 있다.

In [19]:
prompt_partial = prompt.partial(language2="JavaScript")

print(prompt_partial)

input_variables=['language1'] input_types={} partial_variables={'language2': 'JavaScript'} template='{language1}과 {language2}는 각각 누가 만들었나요?'


In [20]:
prompt_partial.invoke("python")

StringPromptValue(text='python과 JavaScript는 각각 누가 만들었나요?')

In [22]:
chain = prompt_partial | llm

print(chain.invoke("Python").content)

Python은 귀도 반 로썸(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다. 그는 이 언어를 만들 때 코드의 가독성과 사용의 용이성을 중시했습니다.

JavaScript는 브렌던 아이크(Brendan Eich)에 의해 1995년에 개발되었습니다. 그는 넷스케이프(Netscape)에서 일할 당시 웹 브라우저에서 동적인 기능을 추가하기 위해 이 언어를 만들었습니다. JavaScript는 웹 개발에서 중요한 역할을 하게 되었고, 현재는 다양한 플랫폼에서도 사용되고 있습니다.


### 파일로부터 template 읽어오기
* prompt를 편하게 작성 및 수정
* 상황에 맞게 파일을 작성해두면 필요할때마다 꺼내서 쓸 수 있다.

In [26]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/language_simple.yml", encoding="UTF-8")

print(prompt)

input_variables=['language'] input_types={} partial_variables={} template='{language} 언어에 대해 3줄로 설명해줘.'


In [27]:
prompt.format(language="Python")

'Python 언어에 대해 3줄로 설명해줘.'

In [28]:
prompt2 = load_prompt("prompts/language.yml", encoding="UTF-8")

print(prompt2.format(language="Python"))

Python 언어에 대해서 설명해줘.
그리고 설명을 다음의 양식에 맞게 정리해줘.
300자 내외.
한글로 작성.
---
#양식
1. 특징
2. 제작자
3. 대표적인 프레임워크
4. 많이 사용되는 분야



In [30]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt2 | ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0) | StrOutputParser()

answer = chain.invoke("Java")

print(answer)

1. 특징: 자바는 객체 지향 프로그래밍 언어로, 플랫폼 독립성을 강조합니다. "한 번 작성하면 어디서나 실행된다"는 슬로건 아래, JVM(Java Virtual Machine)을 통해 다양한 운영체제에서 실행 가능합니다. 또한, 강력한 메모리 관리와 예외 처리 기능을 제공합니다.

2. 제작자: 자바는 1995년 썬 마이크로시스템즈(Sun Microsystems)에서 제임스 고슬링(James Gosling)에 의해 개발되었습니다.

3. 대표적인 프레임워크: 스프링(Spring), 하이버네이트(Hibernate), 자바 서버 페이스스(JavaServer Faces, JSF) 등이 있습니다.

4. 많이 사용되는 분야: 자바는 웹 애플리케이션, 모바일 애플리케이션(안드로이드), 기업용 소프트웨어, 빅데이터 처리 및 클라우드 서비스 등 다양한 분야에서 널리 사용됩니다.


### ChatPromptTemplate
* 대화 목록을 프롬프트로 주입하고자 할 때 사용할 수 있다.
* 메세지는 튜플 형태로 전달
    *("role", "message")로 구성되고 리스트 형태로 생성 가능

**role**
* system : 시스템 설정 메세지로 주로 전역 설정을 할 때 사용
* human : 사용자의 입력 메세지
* ai : AI의 답변 메세지

In [31]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{language}의 제작자는 누구인가요?")

print(chat_prompt)

input_variables=['language'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='{language}의 제작자는 누구인가요?'), additional_kwargs={})]


In [32]:
print(chat_prompt.format(language="Python"))

Human: Python의 제작자는 누구인가요?


In [37]:
chat_template = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "당신은 친절한 인공지능 어시스턴트입니다. 당신의 이름은 {name}입니다."),
    ("human", "반가워요!"),
    ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
    ("human", "{user_input}")
])

messages = chat_template.format_messages(
    name="구찌", user_input="당신의 이름은 무엇입니까?"
)

print(messages)

[SystemMessage(content='당신은 친절한 인공지능 어시스턴트입니다. 당신의 이름은 구찌입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]


In [39]:
print(llm.invoke(messages).content)

제 이름은 구찌입니다! 당신과 대화하게 되어 기쁩니다. 어떤 이야기를 나눌까요?


### MessagePlaceHolder
* 아직 확정된 메세지가 아니지만 나중에 채워질 메세지 위치를 잡아두기 위해 사용한다.
* 보통 대화 기록을 하고싶을 때에 사용한다.

In [41]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate([
    ("system", "당신은 요약 전문 AI 어시스턴트입니다. 당신의 업무는 주요 키워드로 대화를 요약하는 것입니다."),
    MessagesPlaceholder(variable_name="conversation"),
    ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다.")
])

print(chat_prompt)

input_variables=['conversation', 'word_count'] input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.m

In [42]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 업무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?
AI: 아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다.
Human: set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ
AI: 네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [43]:
chain = chat_prompt | llm | StrOutputParser()

In [55]:
input = {
    "word_count": 5,
    "conversation": [
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]    
}

print(chain.invoke(input))

파이썬, 중복 제거, set 사용, 간단 해결, 유용  



In [47]:
!pip install langchain-community

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 34.2 MB/s eta 0:00:00


In [50]:
from langchain_community.llms import Ollama

# model
llm = Ollama(model="gemma2:9b")

response = llm.invoke("지구의 자전 주기는?")

print(response)

 지구의 자전 주기는 약 24시간입니다. 이를 **하루**라고 부릅니다. 하지만 지구의 자전은 완벽하지 않고, 약간씩 변합니다. 또한, 지구가 태양 주위를 도는 회전(공전)도 영향을 미칩니다. 때문에 몇 가지 기준에 따라 다른 시간이 사용될 수 있습니다. 예를 들어:

* **시차**: 지구 자전으로 인해 발생하는 일광은 각 지역에서 다르게 나타나므로, 시각적 관점에서 하루의 길이는 지역에 따라 약간 달라질 수 있습니다.
* **중력과 행성 간 영향**: 태양계의 다른 천체들로부터 오는 중력과 전자기파가 지구 자전 주기에 미세한 영향을 줄 수 있습니다.

즉, 우리가 일상적으로 사용하는 "하루"라는 개념은 실제로 지구 자전 주기를 기반으로 한 근사적인 값이라고 할 수 있습니다.





In [52]:
from langchain.chat_models import ChatOllama

model = ChatOllama(
    model="gemma2:9b",
    temperature=0.1
)

C:\Users\20116\AppData\Local\Temp\ipykernel_13784\2642841664.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(


In [54]:
chain = chat_prompt | model | StrOutputParser()

print(chain.invoke(input))

파이썬, 중복 제거, set 사용, 간단 해결, 유용  

